Import Libraries

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Load text and convert to lower case

In [2]:
filename = "wonderland.txt"
raw_text = open(filename, encoding='UTF-8').read()
raw_text = raw_text.lower()
raw_text

'alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought alice “without pictures or conversations?” so she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a white rabbit with pink eyes ran close by her. there was nothing so very remarkable in that; nor did alice think it so very much out of the way to hear the rabbit say to itself, “oh dear! oh dear! i shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and t

Maping the unique character to integers

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '-': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'a': 11,
 'b': 12,
 'c': 13,
 'd': 14,
 'e': 15,
 'f': 16,
 'g': 17,
 'h': 18,
 'i': 19,
 'j': 20,
 'k': 21,
 'l': 22,
 'm': 23,
 'n': 24,
 'o': 25,
 'p': 26,
 'q': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'x': 34,
 'y': 35,
 'z': 36,
 '—': 37,
 '’': 38,
 '“': 39,
 '”': 40}

Summarize the loaded data

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocabs: ", n_vocab)

Total Characters:  13636
Total Vocabs:  41


Split the dataset into training and testing set

In [5]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append([char_to_int[seq_out]])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  13536


Normalize the data

In [6]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# Normalize
X = X/float(n_vocab)
# One hot encode the output variable
y = np_utils.to_categorical(dataY)

define LSTM model

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Define the checkpoint and Fit the Model

In [ ]:
filepath = "Weights-LSTM-Improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# Fit the model
model.fit(X, y, epochs=1, batch_size=64, callbacks=callbacks_list)

Modualarize All

In [ ]:
# Load larger LSTM network and generate text
import sys
int_to_char = dict((i,c) for i, c in enumerate(chars))
# Summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocabs: ", n_vocab)
# Prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append([char_to_int[seq_out]])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# Reshape X to be [samples, time stamps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# Normalize
X = X/float(n_vocab)
# One hot encode the output variable
y = np_utils.to_categorical(dataY)
# Define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation = 'softmax'))
# Load the network weights
filename = "Weights-LSTM-improvement-44-1.3592-bigger.hdf5"
model.load_weights(filename) 
model.compile(loss='categorical_crossentropy', optimizer='adam')
# Pick the random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed: ")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# Generate characters
for i in range(700):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")